In [1]:
import cv2
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
import heapq
import xlsxwriter
from xlutils.copy import copy    
from xlrd import open_workbook
import os
import math
from scipy import ndimage

In [2]:
# Search all data with _highBW.mat and save to files
path = os.path.abspath("/media/peppermint/easystore/123/Patients")
print(path)
files = []
paths=[];
# Find file with _ute.ni 

for r, d, f in os.walk(path):
    d.sort()
    for file in f:
        if '.mat' in file:
            index=-3;
            name=[];
            find_=False;
            #print(r)
            while(not find_):
                if(file[index]=='.'):
                    find_= True;
                index=index-1;
                if(find_):
                    name.append(file[0:index+1]);
            files.append(name)
            paths.append(r)


/media/peppermint/easystore/123/Patients


In [16]:
excel_ro = open_workbook("/home/peppermint/Desktop/Full_scan.xls")
excel = copy(excel_ro)  # creates a writeable copy
# find the last line
sheet1 = excel.get_sheet(0)  # get a first sheet
tempt_sheet = excel_ro.sheet_by_index(0)
colx = 0
rowx = tempt_sheet.nrows;



for num_file in range(len(files)):
#for num_file in range(10):
    # load data
    try:   
        current_path = paths[num_file]+'/'+str(files[num_file][0])+'.mat';
        #print(current_path)
        mat = scipy.io.loadmat(current_path)
        bar2gas = mat['bar2gas']
        mask_reg = mat['mask_reg']


        total_mean=[];
        for ij in range(128):

            bar2gas_current = ndimage.rotate(bar2gas[ij,:,:],90);
            mask_current = ndimage.rotate(mask_reg[ij,:,:],90);

            if(sum(sum(mask_current))>=900):
                current_img=bar2gas_current ;

                output=cv2.connectedComponentsWithStats(mask_current,4)

                num_labels = output[0];
                labels_im = output[1];
                stats=output[2];
                centroid=output[3];

                if(num_labels<=2):
                    continue

                area=stats[:,4];
                # Delete the background label.
                area=area[1:]

                #Choose the label with largest and second largest except backgrounf
                index_label=np.array(heapq.nlargest(2, range(len(area)), key=area.__getitem__))+1;

                # Find which is left or right
                index_1 = index_label[0]
                index_2 = index_label[1]
                if (centroid[index_1,0]<centroid[index_2,0]):
                    left_label=index_1;
                    right_label=index_2;
                else:
                    left_label=index_2;
                    right_label=index_1;

                # Instant of Do morphology we create distance image 

                dist_transform = cv2.distanceTransform(mask_current,cv2.DIST_L2,3)
                [m,n] = current_img.shape
                sum_left_out=0;
                sum_right_out=0;
                sum_left_in=0;
                sum_right_in=0;

                num_left_out=0;
                num_right_out=0;
                num_left_in=0;
                num_right_in=0;

                for i in range(m):
                    for j in range(n):

                        if(labels_im[i,j]==left_label ):

                            if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                sum_left_out = sum_left_out+ current_img[i,j];
                                num_left_out= num_left_out+1;
                            else: 
                                sum_left_in = sum_left_in+ current_img[i,j];
                                num_left_in= num_left_in+1;


                        elif (labels_im[i,j]==right_label):

                            if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                sum_right_out = sum_right_out+ current_img[i,j];
                                num_right_out= num_right_out+1;
                            else:
                                sum_right_in = sum_right_in+ current_img[i,j];
                                num_right_in= num_right_in+1;


                mean=[];
                have_nan = False;

                if num_left_out!=0:
                    mean.append(sum_left_out/num_left_out);
                else:
                    mean.append(np.nan);
                    have_nan = True;

                if num_left_in!=0:
                    mean.append(sum_left_in/num_left_in);
                else:
                    mean.append(np.nan);
                    have_nan = True;

                if num_right_out!=0:
                    mean.append(sum_right_out/num_right_out);
                else:
                    mean.append(np.nan);
                    have_nan = True;    

                if num_right_in!=0:
                    mean.append(sum_right_in/num_right_in);
                else:
                    mean.append(np.nan);
                    have_nan = True;


                #if(not have_nan):
                #    total_mean.append(mean);
                total_mean.append(mean);

        total_mean=np.array(total_mean);
        total_mean=np.nanmean(total_mean,axis=0)
        #total_mean=np.mean(total_mean,axis=0)

        print(total_mean)
        name = files[num_file][0];
        colx = 0
        try:
            sheet1.write(rowx,colx, name)
            sheet1.write(rowx,colx+1, total_mean[0])
            sheet1.write(rowx,colx+2, total_mean[1])
            sheet1.write(rowx,colx+3, total_mean[2])
            sheet1.write(rowx,colx+4, total_mean[3])
            rowx = rowx+1;
        except:
            print('Something not right')
    except:
        print('Something not right')

excel.save("/home/peppermint/Desktop/Full_scan.xls")


[0.44843124 0.48651465 0.59772321 0.64035961]
[0.49034144 0.48482762 0.58537973 0.61098397]
[0.61570578 0.66219422 0.71046381 0.77967141]
[0.54511071 0.60269788 0.80925437 0.86894214]
[0.38803347 0.4118871  0.459798   0.5135132 ]
[0.53399953 0.5729237  0.64976843 0.67374766]
[0.66934607 0.62849702 0.65596723 0.64033829]
[0. 0. 0. 0.]
[1.16003811 1.26023361 1.31375625 1.33069169]
[1.29006337 1.39922766 1.30743948 1.41153393]
[0.62609169 0.66143093 0.63305358 0.67963262]
[0.68458839 0.75216802 0.71445337 0.7822859 ]
[0.30271268 0.44408645 0.2749762  0.41993407]
[0.27387331 0.49958119 0.3498613  0.52494759]
[0.12545753 0.19970707 0.13415518 0.18627708]
[0.1644806  0.20455302 0.17028162 0.18111429]
[0.09831396 0.14530689 0.09739562 0.17770246]
[0.16894038 0.23039399 0.16528383 0.21721317]
[0.15753475 0.23392608 0.12191709 0.17834256]
[0.26435476 0.26533165 0.22664124 0.24742646]
[0.50579486 0.52343451 0.51298108 0.52309432]
[0.39196065 0.67469131 0.35186457 0.54147226]
[0.79516357 0.865256

[1.35792685 1.39830882 1.26051443 1.29127592]
[1.11570559 1.34199732 1.49658856 1.61258551]
[1.07520782 1.17172952 1.31560841 1.37377762]
[0.92245958 0.95917057 1.09330589 1.12416609]
[0.91815162 0.94599092 1.09056376 1.09386181]
[0.73547045 0.82703861 0.79991439 0.86408705]
[0.57706722 0.69512355 0.59023184 0.65208561]
[0.60745041 0.66132552 0.74085251 0.83561783]
[0.59358    0.68302316 0.80372123 0.87834858]
[0.49686389 0.55718903 0.49474785 0.55691515]
[0.58497822 0.60922823 0.50328359 0.56184882]
[0.79262911 0.78696868 0.67805639 0.77860578]
[0.67074171 0.74564039 0.52230457 0.63237357]
[0.91867175 0.9012522  0.78257198 0.81822673]
[0.87477516 0.89994816 0.75982256 0.90417233]
[0.6862024  0.70184091 0.54463406 0.6452428 ]
[0.6862024  0.70184091 0.54463406 0.6452428 ]
[0.68476263 0.81027429 0.6351876  0.73574346]
[0.7149221  0.89394191 0.61528207 0.81790597]
[0.85050434 0.92385288 0.77451628 0.82831614]
[0.53013992 0.5595222  0.55283185 0.60221378]
[0.5921895  0.63368686 0.66553664 

In [30]:
excel_ro = open_workbook("/home/peppermint/Desktop/Full_scan3.xls")
excel = copy(excel_ro)  # creates a writeable copy
# find the last line
sheet1 = excel.get_sheet(0)  # get a first sheet
tempt_sheet = excel_ro.sheet_by_index(0)
colx = 0
rowx = tempt_sheet.nrows;

# number of split 
ns =3;


for num_file in range(len(files)):
#for num_file in range(1):
# load data
    try:   
        current_path = paths[num_file]+'/'+str(files[num_file][0])+'.mat';
        #print(current_path)
        mat = scipy.io.loadmat(current_path)
        bar2gas = mat['bar2gas']
        mask_reg = mat['mask_reg']


        total_mean=[];
        for ij in range(128):

            bar2gas_current = bar2gas[:,:,ij];
            mask_current = mask_reg[:,:,ij];

            if(sum(sum(mask_current))>=900):
                current_img=bar2gas_current ;

                output=cv2.connectedComponentsWithStats(mask_current,4)

                num_labels = output[0];
                labels_im = output[1];
                stats=output[2];
                centroid=output[3];

                if(num_labels<=2):
                    continue

                area=stats[:,4];
                # Delete the background label.
                area=area[1:]

                #Choose the label with largest and second largest except backgrounf
                index_label=np.array(heapq.nlargest(2, range(len(area)), key=area.__getitem__))+1;

                # Find which is left or right
                index_1 = index_label[0]
                index_2 = index_label[1]
                if (centroid[index_1,0]<centroid[index_2,0]):
                    left_label=index_1;
                    right_label=index_2;
                else:
                    left_label=index_2;
                    right_label=index_1;

                top_left= stats[left_label,1];
                height_left = stats[left_label,3];

                top_right= stats[right_label,1];
                height_right = stats[right_label,3];

                [m,n] = current_img.shape

                sum_left=[0] * ns;
                sum_right=[0] * ns;

                num_left=[0] * ns;
                num_right=[0] * ns;

                for i in range(m):
                    for j in range(n):

                        if(labels_im[i,j]==left_label ):

                            for nlf in range(ns):
                                if(i >= top_left +(height_left/3)*(nlf) and i <= top_left+ (height_left/3)*(nlf+1)):
                                    sum_left[nlf] = sum_left[nlf]+ current_img[i,j];
                                    num_left[nlf]= num_left[nlf]+1;


                        elif (labels_im[i,j]==right_label):
                            for nlf in range(ns):
                                if(i >= top_right +(height_right/3)*(nlf) and i <= top_right+ (height_right/3)*(nlf+1)):
                                    sum_right[nlf] = sum_right[nlf]+ current_img[i,j];
                                    num_right[nlf]= num_right[nlf]+1;


                mean=[];
                have_nan = False;

                for nlf in range(ns):
                    if(num_left[nlf]!=0):
                        mean.append(sum_left[nlf]/num_left[nlf])
                    else:
                        mean.append(np.nan);

                for nlf in range(ns):
                    if(num_right[nlf]!=0):
                        mean.append(sum_right[nlf]/num_right[nlf])
                    else:
                        mean.append(np.nan);


                #if(not have_nan):
                #    total_mean.append(mean);
                total_mean.append(mean);

        total_mean=np.array(total_mean);
        total_mean=np.nanmean(total_mean,axis=0)
        #total_mean=np.mean(total_mean,axis=0)
        print(total_mean)
        name = files[num_file][0];
        colx = 0
        try:
            sheet1.write(rowx,colx, name)
            for nlf in range(ns*2): 
                sheet1.write(rowx,colx+nlf+1, total_mean[nlf])

            rowx = rowx+1;
        except:
            print('Something not right')
    except:
        print('Something not right')

excel.save("/home/peppermint/Desktop/Full_scan3.xls")


[0.52730229 0.49499516 0.462896   0.58987524 0.57198061 0.62038752]
[0.51514327 0.49966577 0.44156603 0.58096284 0.58974875 0.6012695 ]
[0.65117844 0.65826546 0.58382503 0.7233868  0.73301099 0.6687189 ]
[0.65301713 0.6310186  0.61383403 0.80926083 0.79072054 0.81245041]
[0.44588158 0.40120094 0.42297262 0.56780721 0.46012755 0.47650531]
[0.53854298 0.51098282 0.56645278 0.5983913  0.60470829 0.69094613]
[0.75095638 0.59014719 0.60053638 0.71516039 0.67409901 0.73190329]
[0. 0. 0. 0. 0. 0.]
[1.08482935 1.3376538  1.13762905 1.20346542 1.44338665 1.36790317]
[1.22085427 1.45698569 1.10529791 1.2927964  1.55491471 1.30323889]
[0.60060686 0.66265781 0.64127224 0.54916679 0.65083065 0.65965361]
[0.65643738 0.75112724 0.70020677 0.62445155 0.73553658 0.76836637]
[0.3852226  0.39117183 0.30295299 0.39493161 0.38008833 0.278643  ]
[0.37500421 0.40126749 0.38163436 0.4512404  0.45717687 0.41310085]
[0.26541027 0.23880582 0.13329763 0.20468542 0.24580328 0.14896701]
[0.24163486 0.24278436 0.153

[0.80479319 0.79230215 0.73141875 0.84097276 0.80963306 0.67783462]
[0.51704753 0.54409679 0.59940226 0.46760635 0.54038986 0.58565888]
[0.49920631 0.51956416 0.61740671 0.39754082 0.44128725 0.49832025]
[0.51704753 0.54409679 0.59940226 0.46760635 0.54038986 0.58565888]
[0.49920631 0.51956416 0.61740671 0.39754082 0.44128725 0.49832025]
[0.30310848 0.57080301 0.54748189 0.40051464 0.55176212 0.55608659]
[0.80158526 0.75670316 0.69275843 0.76051989 0.6622253  0.45982508]
[0.74899213 0.75512699 0.56582322 0.71267171 0.62758214 0.43924653]
[0.92470022 0.93750012 0.7911276  0.93850854 0.81411822 0.53127117]
[1.00151974 0.98750665 0.87465045 1.09980901 0.98282883 0.67492648]
[0.93568919 0.90864155 0.80400092 0.92142184 0.90215576 0.8033439 ]
[0.86961716 0.84437445 0.75840156 0.85978526 0.84472113 0.72190038]
[0.94263963 0.87237091 0.75251086 0.86125325 0.81146317 0.73138376]
[1.02068266 0.95040817 0.82306487 0.95303762 0.87842019 0.77842578]
[0.99926088 0.90774832 0.79746468 0.85944599 0.8

[0.47334491 0.51470553 0.51499749 0.50325815 0.50223371 0.59158638]
[0.54673424 0.62072734 0.59571338 0.53830988 0.598613   0.47202618]
[0.09356967 0.10884482 0.02676017 0.18156409 0.12599047 0.04345064]
[0.81122514 0.87084009 0.85235367 0.59721323 0.84118143 0.85130674]
[0.55264925 0.6204255  0.50141816 0.54688507 0.58117768 0.5109848 ]
[0.52697696 0.58358922 0.50821656 0.55947103 0.60596811 0.49341209]
[0.82944108 0.73950023 0.75694687 0.79292107 0.78420498 0.75003709]
[0.22506133 0.44348995 0.4186223  0.21643186 0.42600916 0.43677485]
[0.15728243 0.11538671 0.0767888  0.184135   0.10305276 0.13270115]
[0.54461252 0.62298102 0.46936685 0.53994177 0.63004255 0.42107016]
[0.61529641 0.63986987 0.60787301 0.66412618 0.70806705 0.68475668]
[0.5741285  0.46333526 0.34153003 0.68186728 0.51989191 0.35108144]
[0.64907377 0.69565094 0.71384992 0.61523939 0.70253498 0.75562507]
[1.00207261 1.00004173 1.04217099 1.02952396 1.19282273 1.11206205]
[0.93737309 0.95329048 0.9171557  0.93711031 0.9

In [9]:
excel_ro = open_workbook("/home/peppermint/Desktop/NotAll_scan_3split_all_type.xls")
excel = copy(excel_ro)  # creates a writeable copy
# find the last line
sheet1 = excel.get_sheet(0)  # get a first sheet
tempt_sheet = excel_ro.sheet_by_index(0)
colx = 0
rowx = tempt_sheet.nrows;

# number of split 
ns =3;


#for num_file in range(len(files)):
for num_file in range(185):
#for num_file in range(1):
# load data
    try:   
        current_path = paths[num_file]+'/'+str(files[num_file][0])+'.mat';
        #print(current_path)
        mat = scipy.io.loadmat(current_path)
        
        type_of_data=['bar2gas','bar2gas_binning', 'rbc2gas','rbc2gas_binning','gas_highreso_cor','ven_binning'];
        
        number_types=len(type_of_data);
        
        mask_reg = mat['mask_reg']
        
        [rx,ry,rz]= np.array(mask_reg).shape;
        
        data_images=np.zeros([rx,ry,rz,number_types]);
        
        for num,wtype in enumerate(type_of_data): 
            data_images[:,:,:,num]= mat[wtype];
        

        total_mean=[];
        for ij in range(128):
            
            mask_current = mask_reg[:,:,ij];
   

            if(sum(sum(mask_current))>=900):
                

                output=cv2.connectedComponentsWithStats(mask_current,4)

                num_labels = output[0];
                labels_im = output[1];
                stats=output[2];
                centroid=output[3];

                if(num_labels<=2):
                    continue

                area=stats[:,4];
                # Delete the background label.
                area=area[1:]

                #Choose the label with largest and second largest except backgrounf
                index_label=np.array(heapq.nlargest(2, range(len(area)), key=area.__getitem__))+1;

                # Find which is left or right
                index_1 = index_label[0]
                index_2 = index_label[1]
                if (centroid[index_1,0]<centroid[index_2,0]):
                    left_label=index_1;
                    right_label=index_2;
                else:
                    left_label=index_2;
                    right_label=index_1;

                top_left= stats[left_label,1];
                height_left = stats[left_label,3];

                top_right= stats[right_label,1];
                height_right = stats[right_label,3];

                [m,n] = mask_current.shape

                sum_all=[0] * ns * 2 * number_types;

                num_all=[0] * ns * 2 * number_types;


                for i in range(m):
                    for j in range(n):

                        if(labels_im[i,j]==left_label ):
     
                            for nlf in range(ns):
                                if(i >= top_left +(height_left/3)*(nlf) and i <= top_left+ (height_left/3)*(nlf+1)):
                                    
                                    for num in range(number_types):
                                        sum_all[nlf+6*num] = sum_all[nlf+6*num]+ data_images[i,j,ij,num];
                                        num_all[nlf+6*num]= num_all[nlf+6*num]+1;


                        elif (labels_im[i,j]==right_label):
                            for nlf in range(ns):
                                if(i >= top_right +(height_right/3)*(nlf) and i <= top_right+ (height_right/3)*(nlf+1)):
                                    for num in range(number_types):
                                        sum_all[nlf+3+6*num] = sum_all[nlf+3+6*num]+ data_images[i,j,ij,num];
                                        num_all[nlf+3+6*num]= num_all[nlf+3+6*num]+1;


                mean=[];
                have_nan = False;
                for nlf in range(ns * 2 * number_types):
                    if(num_all[nlf]!=0):
                        mean.append(sum_all[nlf]/num_all[nlf])
                    else:
                        mean.append(np.nan);


                #if(not have_nan):
                #    total_mean.append(mean);
                total_mean.append(mean);

        total_mean=np.array(total_mean);
        total_mean=np.nanmean(total_mean,axis=0)
        #total_mean=np.mean(total_mean,axis=0)
        print(total_mean)
        name = files[num_file][0];
        colx = 0
        try:
            sheet1.write(rowx,colx, name)
            for nlf in range(ns * 2 * number_types): 
                sheet1.write(rowx,colx+nlf+1, total_mean[nlf])

            rowx = rowx+1;
        except:
            print('Something not right')
    except:
        print('Something not right')

excel.save("/home/peppermint/Desktop/NotAll_scan_3split_all_type.xls")


[5.27302293e-01 4.94995164e-01 4.62896001e-01 5.89875238e-01
 5.71980611e-01 6.20387517e-01 3.98520699e+00 3.84765907e+00
 3.68353272e+00 4.26175237e+00 4.19770820e+00 4.38850967e+00
 4.13261712e-01 4.08141397e-01 2.50464932e-01 3.81981542e-01
 3.47324557e-01 1.69871587e-01 4.52426592e+00 4.53502074e+00
 3.66629216e+00 4.40081574e+00 4.20690388e+00 3.22955487e+00
 2.21319930e-10 2.34446651e-10 2.37517839e-10 2.35409317e-10
 2.39152015e-10 2.41373086e-10 4.32348072e+00 4.49455574e+00
 4.57432773e+00 4.53320020e+00 4.59104241e+00 4.63235467e+00]


In [14]:
excel_ro = open_workbook("/home/peppermint/Desktop/NotAll_scan_2outin_all_type.xls")
excel = copy(excel_ro)  # creates a writeable copy
# find the last line
sheet1 = excel.get_sheet(0)  # get a first sheet
tempt_sheet = excel_ro.sheet_by_index(0)
colx = 0
rowx = tempt_sheet.nrows;

# number of split 
ns =2;


#for num_file in range(len(files)):
for num_file in range(185):
#for num_file in range(1):
# load data
    try:   
        current_path = paths[num_file]+'/'+str(files[num_file][0])+'.mat';
        #print(current_path)
        mat = scipy.io.loadmat(current_path)

        type_of_data=['bar2gas','bar2gas_binning', 'rbc2gas','rbc2gas_binning','gas_highreso_cor','ven_binning'];

        number_types=len(type_of_data);

        mask_reg = mat['mask_reg']

        [rx,ry,rz]= np.array(mask_reg).shape;

        data_images=np.zeros([rx,ry,rz,number_types]);

        for num,wtype in enumerate(type_of_data): 
            data_images[:,:,:,num]= mat[wtype];


        total_mean=[];
        for ij in range(128):

            mask_current = ndimage.rotate(mask_reg[ij,:,:],90);
            [rx,ry]= np.array(mask_current).shape;

            data_images_current=np.zeros([rx,ry,number_types]);

            for num in range(number_types):
                data_images_current[:,:,num]=ndimage.rotate(data_images[ij,:,:,num],90);


            if(sum(sum(mask_current))>=900):


                output=cv2.connectedComponentsWithStats(mask_current,4)

                num_labels = output[0];
                labels_im = output[1];
                stats=output[2];
                centroid=output[3];

                if(num_labels<=2):
                    continue

                area=stats[:,4];
                # Delete the background label.
                area=area[1:]

                #Choose the label with largest and second largest except backgrounf
                index_label=np.array(heapq.nlargest(2, range(len(area)), key=area.__getitem__))+1;

                # Find which is left or right
                index_1 = index_label[0]
                index_2 = index_label[1]
                if (centroid[index_1,0]<centroid[index_2,0]):
                    left_label=index_1;
                    right_label=index_2;
                else:
                    left_label=index_2;
                    right_label=index_1;

                top_left= stats[left_label,1];
                height_left = stats[left_label,3];

                top_right= stats[right_label,1];
                height_right = stats[right_label,3];

                [m,n] = mask_current.shape

                dist_transform = cv2.distanceTransform(mask_current,cv2.DIST_L2,3)

                sum_all=[0] * ns * 2 * number_types;

                num_all=[0] * ns * 2 * number_types;


                for i in range(m):
                    for j in range(n):

                        if(labels_im[i,j]==left_label ):


                                if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                    nlf = 0;
                                else:
                                    nlf = 1; 


                                for num in range(number_types):
                                    sum_all[nlf+4*num] = sum_all[nlf+4*num]+ data_images_current[i,j,num];
                                    num_all[nlf+4*num]= num_all[nlf+4*num]+1;


                        elif (labels_im[i,j]==right_label):

                                if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                    nlf = 0;
                                else:
                                    nlf = 1; 


                                for num in range(number_types):
                                    sum_all[nlf+2+4*num] = sum_all[nlf+2+4*num]+ data_images_current[i,j,num];
                                    num_all[nlf+2+4*num]= num_all[nlf+2+4*num]+1;


                mean=[];
                have_nan = False;
                for nlf in range(ns * 2 * number_types):
                    if(num_all[nlf]!=0):
                        mean.append(sum_all[nlf]/num_all[nlf])
                    else:
                        mean.append(np.nan);


                #if(not have_nan):
                #    total_mean.append(mean);
                total_mean.append(mean);

        total_mean=np.array(total_mean);
        total_mean=np.nanmean(total_mean,axis=0)
        #total_mean=np.mean(total_mean,axis=0)
        print(total_mean)
        name = files[num_file][0];
        colx = 0
        try:
            sheet1.write(rowx,colx, name)
            for nlf in range(ns * 2 * number_types): 
                sheet1.write(rowx,colx+nlf+1, total_mean[nlf])

            rowx = rowx+1;
        except:
            print('Something not right')
    except:
        print('Something not right')

excel.save("/home/peppermint/Desktop/NotAll_scan_2outin_all_type.xls")


[4.48431237e-01 4.86514654e-01 5.97723210e-01 6.40359612e-01
 3.57457426e+00 3.80495383e+00 4.28520416e+00 4.52453584e+00
 2.68704545e-01 2.93503071e-01 2.62502692e-01 2.50674661e-01
 3.66262260e+00 3.91316701e+00 3.72794455e+00 3.71129427e+00
 2.03024621e-10 2.51929370e-10 2.21247952e-10 2.69618093e-10
 4.08316289e+00 4.80295924e+00 4.32396843e+00 5.09748653e+00]


In [21]:
excel_ro = open_workbook("/home/peppermint/Desktop/NotAll_scan_2outin_3cut_all_type.xls")
excel = copy(excel_ro)  # creates a writeable copy
# find the last line
sheet1 = excel.get_sheet(0)  # get a first sheet
tempt_sheet = excel_ro.sheet_by_index(0)
colx = 0
rowx = tempt_sheet.nrows;

# number of split 
ns = 2*3;
number_cut=3;

#for num_file in range(len(files)):
for num_file in range(185):
#for num_file in range(1):
# load data
    try:   
        current_path = paths[num_file]+'/'+str(files[num_file][0])+'.mat';
        #print(current_path)
        mat = scipy.io.loadmat(current_path)

        type_of_data=['bar2gas','bar2gas_binning', 'rbc2gas','rbc2gas_binning','gas_highreso_cor','ven_binning'];

        number_types=len(type_of_data);

        mask_reg = mat['mask_reg']

        [rx,ry,rz]= np.array(mask_reg).shape;

        data_images=np.zeros([rx,ry,rz,number_types]);

        for num,wtype in enumerate(type_of_data): 
            data_images[:,:,:,num]= mat[wtype];


        total_mean=[];
        for ij in range(128):

            mask_current = ndimage.rotate(mask_reg[ij,:,:],90);
            [rx,ry]= np.array(mask_current).shape;

            data_images_current=np.zeros([rx,ry,number_types]);

            for num in range(number_types):
                data_images_current[:,:,num]=ndimage.rotate(data_images[ij,:,:,num],90);


            if(sum(sum(mask_current))>=900):


                output=cv2.connectedComponentsWithStats(mask_current,4)

                num_labels = output[0];
                labels_im = output[1];
                stats=output[2];
                centroid=output[3];

                if(num_labels<=2):
                    continue

                area=stats[:,4];
                # Delete the background label.
                area=area[1:]

                #Choose the label with largest and second largest except backgrounf
                index_label=np.array(heapq.nlargest(2, range(len(area)), key=area.__getitem__))+1;

                # Find which is left or right
                index_1 = index_label[0]
                index_2 = index_label[1]
                if (centroid[index_1,0]<centroid[index_2,0]):
                    left_label=index_1;
                    right_label=index_2;
                else:
                    left_label=index_2;
                    right_label=index_1;

                top_left= stats[left_label,1];
                height_left = stats[left_label,3];

                top_right= stats[right_label,1];
                height_right = stats[right_label,3];

                [m,n] = mask_current.shape

                dist_transform = cv2.distanceTransform(mask_current,cv2.DIST_L2,3)

                sum_all=[0] * ns * 2 * number_types;

                num_all=[0] * ns * 2 * number_types;


                for i in range(m):
                    for j in range(n):

                        if(labels_im[i,j]==left_label ):


                                if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                    nlf = 0;
                                else:
                                    nlf = 1; 
                                for ncut in range(number_cut):
                                    if(i >= top_left +(height_left/3)*(ncut) and i <= top_left+ (height_left/3)*(ncut+1)):
                                        nlf_c = nlf + 2*ncut;
                                        for num in range(number_types):
                                            sum_all[nlf_c+12*num] = sum_all[nlf_c+12*num]+ data_images_current[i,j,num];
                                            num_all[nlf_c+12*num]= num_all[nlf_c+12*num]+1;


                        elif (labels_im[i,j]==right_label):

                                if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                    nlf = 0;
                                else:
                                    nlf = 1; 

                                for ncut in range(number_cut):
                                    if(i >= top_left +(height_left/3)*(ncut) and i <= top_left+ (height_left/3)*(ncut+1)):
                                        nlf_c = nlf + 2*ncut;
                                        for num in range(number_types):
                                            sum_all[nlf_c+6+12*num] = sum_all[nlf_c+6+12*num]+ data_images_current[i,j,num];
                                            num_all[nlf_c+6+12*num]= num_all[nlf_c+6+12*num]+1;



                mean=[];
                have_nan = False;
                for nlf in range(ns * 2 * number_types):
                    if(num_all[nlf]!=0):
                        mean.append(sum_all[nlf]/num_all[nlf])
                    else:
                        mean.append(np.nan);


                #if(not have_nan):
                #    total_mean.append(mean);
                total_mean.append(mean);

        total_mean=np.array(total_mean);
        total_mean=np.nanmean(total_mean,axis=0)
        #total_mean=np.mean(total_mean,axis=0)
        print(total_mean)
        name = files[num_file][0];
        colx = 0
        try:
            sheet1.write(rowx,colx, name)
            for nlf in range(ns * 2 * number_types): 
                sheet1.write(rowx,colx+nlf+1, total_mean[nlf])

            rowx = rowx+1;
        except:
            print('Something not right')
    except:
        print('Something not right')

excel.save("/home/peppermint/Desktop/NotAll_scan_2outin_3cut_all_type.xls")

[5.48877919e-01 5.54475542e-01 4.18639854e-01 4.61321102e-01
 3.78495549e-01 4.28931721e-01 6.71149692e-01 6.90895145e-01
 6.04045853e-01 6.15294449e-01 5.00875345e-01 5.31284894e-01
 4.05432963e+00 4.11382953e+00 3.44488165e+00 3.70651651e+00
 3.23393516e+00 3.54197030e+00 4.62835855e+00 4.79994259e+00
 4.30080927e+00 4.35222369e+00 3.83743067e+00 3.99106111e+00
 3.46646956e-01 3.52451755e-01 2.98741800e-01 3.13084423e-01
 1.79869766e-01 1.99065410e-01 2.93116100e-01 2.67170852e-01
 2.70044789e-01 2.63245796e-01 1.93589973e-01 1.59950443e-01
 4.16870225e+00 4.23606913e+00 3.82136438e+00 4.02095818e+00
 3.11745149e+00 3.39768357e+00 3.92516621e+00 3.81118749e+00
 3.79028803e+00 3.76895336e+00 3.29531221e+00 3.13503402e+00
 2.15301248e-10 2.53674663e-10 1.93299500e-10 2.53956424e-10
 2.01045613e-10 2.54234594e-10 2.20215153e-10 2.69433295e-10
 2.25547978e-10 2.63283474e-10 2.23384974e-10 2.85019439e-10
 4.19492530e+00 4.79604614e+00 3.96670051e+00 4.86040124e+00
 4.11296987e+00 4.895825

In [ ]:
# Just copy above 3 code cause I am to lazy

In [3]:
excel_ro = open_workbook("/home/peppermint/Desktop/NotAll_scan_3split_all_type.xls")
excel = copy(excel_ro)  # creates a writeable copy
# find the last line
sheet1 = excel.get_sheet(0)  # get a first sheet
tempt_sheet = excel_ro.sheet_by_index(0)
colx = 0
rowx = tempt_sheet.nrows;

# number of split 
ns =3;


#for num_file in range(len(files)):
for num_file in range(210):
#for num_file in range(1):
# load data
    try:   
        current_path = paths[num_file]+'/'+str(files[num_file][0])+'.mat';
        #print(current_path)
        mat = scipy.io.loadmat(current_path)
        
        type_of_data=['bar2gas','bar2gas_binning', 'rbc2gas','rbc2gas_binning','gas_highreso_cor','ven_binning'];
        
        number_types=len(type_of_data);
        
        mask_reg = mat['mask_reg']
        
        [rx,ry,rz]= np.array(mask_reg).shape;
        
        data_images=np.zeros([rx,ry,rz,number_types]);
        
        for num,wtype in enumerate(type_of_data): 
            data_images[:,:,:,num]= mat[wtype];
        

        total_mean=[];
        for ij in range(128):
            
            mask_current = mask_reg[:,:,ij];
   

            if(sum(sum(mask_current))>=900):
                

                output=cv2.connectedComponentsWithStats(mask_current,4)

                num_labels = output[0];
                labels_im = output[1];
                stats=output[2];
                centroid=output[3];

                if(num_labels<=2):
                    continue

                area=stats[:,4];
                # Delete the background label.
                area=area[1:]

                #Choose the label with largest and second largest except backgrounf
                index_label=np.array(heapq.nlargest(2, range(len(area)), key=area.__getitem__))+1;

                # Find which is left or right
                index_1 = index_label[0]
                index_2 = index_label[1]
                if (centroid[index_1,0]<centroid[index_2,0]):
                    left_label=index_1;
                    right_label=index_2;
                else:
                    left_label=index_2;
                    right_label=index_1;

                top_left= stats[left_label,1];
                height_left = stats[left_label,3];

                top_right= stats[right_label,1];
                height_right = stats[right_label,3];

                [m,n] = mask_current.shape

                sum_all=[0] * ns * 2 * number_types;

                num_all=[0] * ns * 2 * number_types;


                for i in range(m):
                    for j in range(n):

                        if(labels_im[i,j]==left_label ):
     
                            for nlf in range(ns):
                                if(i >= top_left +(height_left/3)*(nlf) and i <= top_left+ (height_left/3)*(nlf+1)):
                                    
                                    for num in range(number_types):
                                        sum_all[nlf+6*num] = sum_all[nlf+6*num]+ data_images[i,j,ij,num];
                                        num_all[nlf+6*num]= num_all[nlf+6*num]+1;


                        elif (labels_im[i,j]==right_label):
                            for nlf in range(ns):
                                if(i >= top_right +(height_right/3)*(nlf) and i <= top_right+ (height_right/3)*(nlf+1)):
                                    for num in range(number_types):
                                        sum_all[nlf+3+6*num] = sum_all[nlf+3+6*num]+ data_images[i,j,ij,num];
                                        num_all[nlf+3+6*num]= num_all[nlf+3+6*num]+1;


                mean=[];
                have_nan = False;
                for nlf in range(ns * 2 * number_types):
                    if(num_all[nlf]!=0):
                        mean.append(sum_all[nlf]/num_all[nlf])
                    else:
                        mean.append(np.nan);


                #if(not have_nan):
                #    total_mean.append(mean);
                total_mean.append(mean);

        total_mean=np.array(total_mean);
        total_mean=np.nanmean(total_mean,axis=0)
        #total_mean=np.mean(total_mean,axis=0)
        print(total_mean)
        name = files[num_file][0];
        colx = 0
        try:
            sheet1.write(rowx,colx, name)
            for nlf in range(ns * 2 * number_types): 
                sheet1.write(rowx,colx+nlf+1, total_mean[nlf])

            rowx = rowx+1;
        except:
            print('Something not right')
    except:
        print('Something not right')

excel.save("/home/peppermint/Desktop/NotAll_scan_3split_all_type.xls")


# Second run



excel_ro = open_workbook("/home/peppermint/Desktop/NotAll_scan_2outin_all_type.xls")
excel = copy(excel_ro)  # creates a writeable copy
# find the last line
sheet1 = excel.get_sheet(0)  # get a first sheet
tempt_sheet = excel_ro.sheet_by_index(0)
colx = 0
rowx = tempt_sheet.nrows;

# number of split 
ns =2;


#for num_file in range(len(files)):
for num_file in range(210):
#for num_file in range(1):
# load data
    try:   
        current_path = paths[num_file]+'/'+str(files[num_file][0])+'.mat';
        #print(current_path)
        mat = scipy.io.loadmat(current_path)

        type_of_data=['bar2gas','bar2gas_binning', 'rbc2gas','rbc2gas_binning','gas_highreso_cor','ven_binning'];

        number_types=len(type_of_data);

        mask_reg = mat['mask_reg']

        [rx,ry,rz]= np.array(mask_reg).shape;

        data_images=np.zeros([rx,ry,rz,number_types]);

        for num,wtype in enumerate(type_of_data): 
            data_images[:,:,:,num]= mat[wtype];


        total_mean=[];
        for ij in range(128):

            mask_current = ndimage.rotate(mask_reg[ij,:,:],90);
            [rx,ry]= np.array(mask_current).shape;

            data_images_current=np.zeros([rx,ry,number_types]);

            for num in range(number_types):
                data_images_current[:,:,num]=ndimage.rotate(data_images[ij,:,:,num],90);


            if(sum(sum(mask_current))>=900):


                output=cv2.connectedComponentsWithStats(mask_current,4)

                num_labels = output[0];
                labels_im = output[1];
                stats=output[2];
                centroid=output[3];

                if(num_labels<=2):
                    continue

                area=stats[:,4];
                # Delete the background label.
                area=area[1:]

                #Choose the label with largest and second largest except backgrounf
                index_label=np.array(heapq.nlargest(2, range(len(area)), key=area.__getitem__))+1;

                # Find which is left or right
                index_1 = index_label[0]
                index_2 = index_label[1]
                if (centroid[index_1,0]<centroid[index_2,0]):
                    left_label=index_1;
                    right_label=index_2;
                else:
                    left_label=index_2;
                    right_label=index_1;

                top_left= stats[left_label,1];
                height_left = stats[left_label,3];

                top_right= stats[right_label,1];
                height_right = stats[right_label,3];

                [m,n] = mask_current.shape

                dist_transform = cv2.distanceTransform(mask_current,cv2.DIST_L2,3)

                sum_all=[0] * ns * 2 * number_types;

                num_all=[0] * ns * 2 * number_types;


                for i in range(m):
                    for j in range(n):

                        if(labels_im[i,j]==left_label ):


                                if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                    nlf = 0;
                                else:
                                    nlf = 1; 


                                for num in range(number_types):
                                    sum_all[nlf+4*num] = sum_all[nlf+4*num]+ data_images_current[i,j,num];
                                    num_all[nlf+4*num]= num_all[nlf+4*num]+1;


                        elif (labels_im[i,j]==right_label):

                                if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                    nlf = 0;
                                else:
                                    nlf = 1; 


                                for num in range(number_types):
                                    sum_all[nlf+2+4*num] = sum_all[nlf+2+4*num]+ data_images_current[i,j,num];
                                    num_all[nlf+2+4*num]= num_all[nlf+2+4*num]+1;


                mean=[];
                have_nan = False;
                for nlf in range(ns * 2 * number_types):
                    if(num_all[nlf]!=0):
                        mean.append(sum_all[nlf]/num_all[nlf])
                    else:
                        mean.append(np.nan);


                #if(not have_nan):
                #    total_mean.append(mean);
                total_mean.append(mean);

        total_mean=np.array(total_mean);
        total_mean=np.nanmean(total_mean,axis=0)
        #total_mean=np.mean(total_mean,axis=0)
        print(total_mean)
        name = files[num_file][0];
        colx = 0
        try:
            sheet1.write(rowx,colx, name)
            for nlf in range(ns * 2 * number_types): 
                sheet1.write(rowx,colx+nlf+1, total_mean[nlf])

            rowx = rowx+1;
        except:
            print('Something not right')
    except:
        print('Something not right')

excel.save("/home/peppermint/Desktop/NotAll_scan_2outin_all_type.xls")


# Third run 


excel_ro = open_workbook("/home/peppermint/Desktop/NotAll_scan_2outin_3cut_all_type.xls")
excel = copy(excel_ro)  # creates a writeable copy
# find the last line
sheet1 = excel.get_sheet(0)  # get a first sheet
tempt_sheet = excel_ro.sheet_by_index(0)
colx = 0
rowx = tempt_sheet.nrows;

# number of split 
ns = 2*3;
number_cut=3;

#for num_file in range(len(files)):
for num_file in range(210):
#for num_file in range(1):
# load data
    try:   
        current_path = paths[num_file]+'/'+str(files[num_file][0])+'.mat';
        #print(current_path)
        mat = scipy.io.loadmat(current_path)

        type_of_data=['bar2gas','bar2gas_binning', 'rbc2gas','rbc2gas_binning','gas_highreso_cor','ven_binning'];

        number_types=len(type_of_data);

        mask_reg = mat['mask_reg']

        [rx,ry,rz]= np.array(mask_reg).shape;

        data_images=np.zeros([rx,ry,rz,number_types]);

        for num,wtype in enumerate(type_of_data): 
            data_images[:,:,:,num]= mat[wtype];


        total_mean=[];
        for ij in range(128):

            mask_current = ndimage.rotate(mask_reg[ij,:,:],90);
            [rx,ry]= np.array(mask_current).shape;

            data_images_current=np.zeros([rx,ry,number_types]);

            for num in range(number_types):
                data_images_current[:,:,num]=ndimage.rotate(data_images[ij,:,:,num],90);


            if(sum(sum(mask_current))>=900):


                output=cv2.connectedComponentsWithStats(mask_current,4)

                num_labels = output[0];
                labels_im = output[1];
                stats=output[2];
                centroid=output[3];

                if(num_labels<=2):
                    continue

                area=stats[:,4];
                # Delete the background label.
                area=area[1:]

                #Choose the label with largest and second largest except backgrounf
                index_label=np.array(heapq.nlargest(2, range(len(area)), key=area.__getitem__))+1;

                # Find which is left or right
                index_1 = index_label[0]
                index_2 = index_label[1]
                if (centroid[index_1,0]<centroid[index_2,0]):
                    left_label=index_1;
                    right_label=index_2;
                else:
                    left_label=index_2;
                    right_label=index_1;

                top_left= stats[left_label,1];
                height_left = stats[left_label,3];

                top_right= stats[right_label,1];
                height_right = stats[right_label,3];

                [m,n] = mask_current.shape

                dist_transform = cv2.distanceTransform(mask_current,cv2.DIST_L2,3)

                sum_all=[0] * ns * 2 * number_types;

                num_all=[0] * ns * 2 * number_types;


                for i in range(m):
                    for j in range(n):

                        if(labels_im[i,j]==left_label ):


                                if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                    nlf = 0;
                                else:
                                    nlf = 1; 
                                for ncut in range(number_cut):
                                    if(i >= top_left +(height_left/3)*(ncut) and i <= top_left+ (height_left/3)*(ncut+1)):
                                        nlf_c = nlf + 2*ncut;
                                        for num in range(number_types):
                                            sum_all[nlf_c+12*num] = sum_all[nlf_c+12*num]+ data_images_current[i,j,num];
                                            num_all[nlf_c+12*num]= num_all[nlf_c+12*num]+1;


                        elif (labels_im[i,j]==right_label):

                                if((dist_transform[i,j]>0 and dist_transform[i,j]<=3)):
                                    nlf = 0;
                                else:
                                    nlf = 1; 

                                for ncut in range(number_cut):
                                    if(i >= top_left +(height_left/3)*(ncut) and i <= top_left+ (height_left/3)*(ncut+1)):
                                        nlf_c = nlf + 2*ncut;
                                        for num in range(number_types):
                                            sum_all[nlf_c+6+12*num] = sum_all[nlf_c+6+12*num]+ data_images_current[i,j,num];
                                            num_all[nlf_c+6+12*num]= num_all[nlf_c+6+12*num]+1;



                mean=[];
                have_nan = False;
                for nlf in range(ns * 2 * number_types):
                    if(num_all[nlf]!=0):
                        mean.append(sum_all[nlf]/num_all[nlf])
                    else:
                        mean.append(np.nan);


                #if(not have_nan):
                #    total_mean.append(mean);
                total_mean.append(mean);

        total_mean=np.array(total_mean);
        total_mean=np.nanmean(total_mean,axis=0)
        #total_mean=np.mean(total_mean,axis=0)
        print(total_mean)
        name = files[num_file][0];
        colx = 0
        try:
            sheet1.write(rowx,colx, name)
            for nlf in range(ns * 2 * number_types): 
                sheet1.write(rowx,colx+nlf+1, total_mean[nlf])

            rowx = rowx+1;
        except:
            print('Something not right')
    except:
        print('Something not right')

excel.save("/home/peppermint/Desktop/NotAll_scan_2outin_3cut_all_type.xls")

















[5.27302293e-01 4.94995164e-01 4.62896001e-01 5.89875238e-01
 5.71980611e-01 6.20387517e-01 3.98520699e+00 3.84765907e+00
 3.68353272e+00 4.26175237e+00 4.19770820e+00 4.38850967e+00
 4.13261712e-01 4.08141397e-01 2.50464932e-01 3.81981542e-01
 3.47324557e-01 1.69871587e-01 4.52426592e+00 4.53502074e+00
 3.66629216e+00 4.40081574e+00 4.20690388e+00 3.22955487e+00
 2.21319930e-10 2.34446651e-10 2.37517839e-10 2.35409317e-10
 2.39152015e-10 2.41373086e-10 4.32348072e+00 4.49455574e+00
 4.57432773e+00 4.53320020e+00 4.59104241e+00 4.63235467e+00]
[5.15143270e-01 4.99665767e-01 4.41566027e-01 5.80962845e-01
 5.89748749e-01 6.01269502e-01 3.91695060e+00 3.87035346e+00
 3.53723077e+00 4.22882302e+00 4.29642570e+00 4.28103986e+00
 3.88926288e-01 3.58120668e-01 2.21611622e-01 3.91343873e-01
 3.28970564e-01 1.57738500e-01 4.36960326e+00 4.27771664e+00
 3.40530458e+00 4.43989288e+00 4.11342199e+00 3.07776387e+00
 2.64337330e-10 2.90086352e-10 2.64538204e-10 2.84871235e-10
 2.93830917e-10 2.70354

[2.41634858e-01 2.42784356e-01 1.53509218e-01 2.05219973e-01
 2.16484539e-01 1.70082930e-01 2.41037886e+00 2.50553906e+00
 1.98668367e+00 2.17077472e+00 2.31359303e+00 2.00650039e+00
 1.77226758e-01 1.84549914e-01 8.86418734e-02 1.56645023e-01
 1.58648586e-01 8.20542409e-02 2.57148824e+00 2.81809293e+00
 2.00155766e+00 2.24961803e+00 2.52087651e+00 1.90785092e+00
 4.51508231e-11 6.22192924e-11 5.05547559e-11 4.11997391e-11
 5.08804710e-11 4.97709115e-11 2.64443881e+00 3.08896662e+00
 2.81766398e+00 2.55449506e+00 2.79516137e+00 2.79482483e+00]
[2.07703008e-01 2.09073474e-01 1.05133371e-01 2.26331230e-01
 2.22344095e-01 1.25808969e-01 2.27070103e+00 2.38199425e+00
 1.71898600e+00 2.38659534e+00 2.40689238e+00 1.81368152e+00
 1.46612001e-01 1.57710135e-01 5.47189152e-02 1.56893841e-01
 1.32604223e-01 4.31575121e-02 2.48036723e+00 2.73687546e+00
 1.76673212e+00 2.63888004e+00 2.61085438e+00 1.72774832e+00
 7.03564776e-11 8.82207551e-11 5.71582293e-11 7.38858915e-11
 8.55776557e-11 6.50455

[7.94400890e-01 7.60132264e-01 7.21915258e-01 9.15182781e-01
 9.59317655e-01 9.38970430e-01 5.15996100e+00 5.02047902e+00
 4.80181976e+00 5.73468941e+00 5.91682343e+00 5.84011159e+00
 1.49709597e-01 1.67552699e-01 8.14644526e-02 1.20667744e-01
 1.39780864e-01 4.86305851e-02 3.00306116e+00 3.15109300e+00
 2.57171242e+00 2.90155658e+00 3.02246713e+00 2.36650398e+00
 2.97321069e-10 3.07582358e-10 3.07298902e-10 3.25084090e-10
 3.42479213e-10 3.23175988e-10 4.43486360e+00 4.54180387e+00
 4.58161235e+00 4.69900372e+00 4.92899016e+00 4.70672755e+00]
[5.79469743e-01 6.91681213e-01 6.04259203e-01 5.72569745e-01
 6.98855196e-01 6.40382471e-01 4.07804330e+00 4.69609052e+00
 4.15239229e+00 4.07892029e+00 4.71857384e+00 4.35027955e+00
 4.75155569e-01 4.38164309e-01 2.76959529e-01 4.36288387e-01
 3.87413811e-01 1.97870017e-01 4.30526202e+00 4.54041040e+00
 3.30733087e+00 4.08439186e+00 4.22816449e+00 3.01315433e+00
 9.22998805e-11 1.29423645e-10 8.52001451e-11 9.68496901e-11
 1.21441077e-10 9.51513

[7.87006363e-01 7.78107222e-01 7.76500362e-01 8.01137515e-01
 8.69505359e-01 1.01424384e+00 4.72952193e+00 4.67256766e+00
 4.63252494e+00 4.72259232e+00 4.94703008e+00 5.46211542e+00
 1.93069335e-01 1.72047369e-01 6.26276132e-02 2.27755411e-01
 1.80357022e-01 3.15945273e-02 3.17263283e+00 3.05598583e+00
 2.41048890e+00 3.39428412e+00 3.10698903e+00 2.20723947e+00
 2.66666436e-10 2.77216861e-10 2.64763639e-10 2.84726806e-10
 2.81576906e-10 2.61797374e-10 4.66674692e+00 4.80093784e+00
 4.70236849e+00 4.90938087e+00 4.87718845e+00 4.66203533e+00]
[9.34462466e-01 9.95978350e-01 7.78387180e-01 8.94854217e-01
 1.03436433e+00 9.11309915e-01 5.15049414e+00 5.45998561e+00
 4.61949237e+00 5.00681724e+00 5.59864478e+00 5.07711929e+00
 9.41660501e-02 1.25829155e-01 5.46769515e-02 8.32364182e-02
 8.91502502e-02 2.84982534e-02 2.49113402e+00 2.75926356e+00
 2.31698904e+00 2.41607479e+00 2.57780876e+00 2.12074619e+00
 3.60526203e-10 3.99966916e-10 4.01690894e-10 3.44902340e-10
 4.36246469e-10 3.88451

[1.03308707e+00 8.68714058e-01 8.02884251e-01 9.32213911e-01
 8.60571809e-01 9.53645639e-01 5.56028496e+00 4.98146793e+00
 4.72755312e+00 5.19690047e+00 4.96469238e+00 5.24513650e+00
 2.36395610e-01 2.37087060e-01 1.25543531e-01 2.96474219e-01
 2.48535202e-01 7.73613539e-02 3.36036346e+00 3.37355348e+00
 2.73763126e+00 3.70179614e+00 3.46247798e+00 2.45262718e+00
 2.10711011e-10 2.27325015e-10 2.15876027e-10 2.21594115e-10
 2.26409036e-10 2.14379882e-10 4.19917994e+00 4.44044300e+00
 4.31460828e+00 4.36688569e+00 4.40672030e+00 4.28259499e+00]
[8.51018407e-01 8.49853603e-01 7.95349944e-01 8.18068281e-01
 8.26760703e-01 8.34829092e-01 4.88344978e+00 4.89602533e+00
 4.58946415e+00 4.70385153e+00 4.78315546e+00 4.71274114e+00
 1.87400645e-01 1.75877634e-01 7.84711838e-02 1.77185991e-01
 1.72380818e-01 6.16601578e-02 3.06768469e+00 3.07802959e+00
 2.35242436e+00 3.01270898e+00 3.03496518e+00 2.22789252e+00
 3.43311999e-10 4.54508342e-10 3.45700151e-10 3.67664618e-10
 4.40311478e-10 3.39259

...

[7.97323133e-01 8.22579312e-01 6.00818654e-01 5.99855503e-01
 5.28846610e-01 5.76806143e-01 7.47820952e-01 7.90830087e-01
 6.07239323e-01 6.84722682e-01 4.24007385e-01 4.93768940e-01
 4.63888766e+00 4.78903727e+00 3.78595955e+00 3.92142752e+00
 3.59705620e+00 3.86232171e+00 4.47960405e+00 4.71600847e+00
 3.89158300e+00 4.29939030e+00 3.25409593e+00 3.57367064e+00
 1.85332020e-01 2.54466997e-01 2.35025801e-01 2.68620573e-01
 1.75031044e-01 1.86623005e-01 1.80332568e-01 2.19498569e-01
 2.11957494e-01 2.41407689e-01 1.30040589e-01 1.46806292e-01
 2.97354636e+00 3.44224928e+00 3.00070978e+00 3.38963298e+00
 2.75897345e+00 2.99545606e+00 2.98298764e+00 3.29210412e+00
 3.03982317e+00 3.38978931e+00 2.62993812e+00 2.83268797e+00
 2.02548240e-10 2.56093257e-10 1.55872921e-10 1.97285159e-10
 1.89293911e-10 2.34302124e-10 2.14255147e-10 2.74433134e-10
 1.81750015e-10 2.31268726e-10 1.97912912e-10 2.37767698e-10
 3.71205662e+00 4.27235399e+00 3.22839647e+00 3.65175223e+00
 3.57664106e+00 4.073034

[1.64025204e+00 1.70322118e+00 1.47494232e+00 1.54555777e+00
 1.26592627e+00 1.31589093e+00 1.65731131e+00 1.74691145e+00
 1.64897294e+00 1.68330045e+00 1.48745990e+00 1.51336728e+00
 7.62958171e+00 7.92476229e+00 7.08447989e+00 7.38032880e+00
 6.33754624e+00 6.54710016e+00 7.64613624e+00 7.99304354e+00
 7.60102130e+00 7.80604129e+00 7.13897786e+00 7.23620576e+00
 1.76732241e-01 1.98255181e-01 2.97670203e-01 2.81483812e-01
 1.77209661e-01 1.61158553e-01 2.38803594e-01 2.59478374e-01
 3.35164261e-01 3.56625714e-01 1.74682470e-01 1.91113911e-01
 2.99892549e+00 3.17206622e+00 3.60498421e+00 3.60712772e+00
 2.96816687e+00 2.98948260e+00 3.30969798e+00 3.47057140e+00
 3.76275482e+00 3.92575479e+00 2.97278974e+00 3.13780071e+00
 1.48761800e-10 1.85334336e-10 1.36893873e-10 1.69613808e-10
 1.31973996e-10 1.90905753e-10 1.55180395e-10 1.92525046e-10
 1.48776849e-10 1.73579661e-10 1.61199919e-10 1.75142215e-10
 4.09253781e+00 4.78261133e+00 3.91745639e+00 4.48201788e+00
 3.81597107e+00 4.926699

[7.74501462e-01 8.82024266e-01 8.00785374e-01 8.74950537e-01
 5.73238608e-01 6.39293577e-01 9.63441041e-01 1.02285998e+00
 8.71492001e-01 9.27900975e-01 7.23503045e-01 7.18465315e-01
 5.01338716e+00 5.57506189e+00 5.15136363e+00 5.54291657e+00
 4.07958594e+00 4.44719095e+00 5.80334267e+00 6.17976622e+00
 5.31048103e+00 5.74406052e+00 4.58732690e+00 4.77995007e+00
 1.19189436e-01 1.09217837e-01 2.19416769e-01 1.83385900e-01
 2.19624471e-01 1.74383596e-01 1.50245717e-01 1.60278733e-01
 3.48501066e-01 2.91861325e-01 3.30109534e-01 2.22103513e-01
 2.70186067e+00 2.81675375e+00 3.29509908e+00 3.24511499e+00
 3.10510136e+00 3.04870175e+00 2.88173250e+00 3.04966868e+00
 3.69535592e+00 3.75202659e+00 3.28195934e+00 3.27987989e+00
 8.58615942e-11 1.10814384e-10 8.70107595e-11 1.07845666e-10
 8.16795228e-11 9.87444459e-11 9.18527906e-11 1.09841031e-10
 7.98477172e-11 9.48769270e-11 7.78497374e-11 9.73136914e-11
 3.57665508e+00 4.22625697e+00 3.60013402e+00 4.15043400e+00
 3.47818559e+00 3.934261

[1.17234116e+00 1.07970682e+00 8.78699658e-01 8.60681250e-01
 6.53108627e-01 7.05613067e-01 9.62384566e-01 9.45681988e-01
 7.87616542e-01 8.35817007e-01 6.86218825e-01 7.08340195e-01
 6.06301368e+00 5.73664809e+00 4.94818377e+00 4.95368606e+00
 4.05567821e+00 4.33264890e+00 5.30081346e+00 5.25375557e+00
 4.64422776e+00 4.86711975e+00 4.28990437e+00 4.37482705e+00
 3.05527219e-01 3.95875516e-01 3.66102647e-01 4.13309863e-01
 3.06312100e-01 3.19117328e-01 3.31558838e-01 3.66443697e-01
 3.80407006e-01 3.81830588e-01 2.70899394e-01 2.69385043e-01
 3.73656660e+00 4.20910953e+00 3.92710214e+00 4.28016884e+00
 3.37623644e+00 3.73237361e+00 3.83989896e+00 4.05944974e+00
 4.06127695e+00 4.14012789e+00 3.51622513e+00 3.57514621e+00
 1.45246671e-10 1.58364906e-10 1.21729047e-10 1.56732617e-10
 1.02840333e-10 1.41145290e-10 1.34622369e-10 1.62343379e-10
 1.26789866e-10 1.59486041e-10 1.34394301e-10 1.59212731e-10
 4.59447357e+00 4.89059625e+00 4.08642039e+00 4.86541594e+00
 3.71738309e+00 4.561417

[6.96318257e-01 7.08393891e-01 5.45613209e-01 6.17538776e-01
 5.37059037e-01 5.65976784e-01 7.69334153e-01 7.76653020e-01
 6.61054652e-01 6.71765136e-01 5.27513135e-01 5.45751325e-01
 4.32517454e+00 4.37461161e+00 3.80249152e+00 4.06438692e+00
 3.82642517e+00 3.94557906e+00 4.63272456e+00 4.66743607e+00
 4.22912085e+00 4.28132590e+00 3.76949967e+00 3.85585321e+00
 4.58881872e-01 4.77592552e-01 4.75839017e-01 5.05589419e-01
 2.48093910e-01 2.76621348e-01 4.36152583e-01 4.35074480e-01
 4.43225990e-01 4.37318253e-01 2.31021985e-01 2.71386499e-01
 4.50497128e+00 4.61045725e+00 4.54513424e+00 4.71658451e+00
 3.46007451e+00 3.61701623e+00 4.40682190e+00 4.42632088e+00
 4.43512566e+00 4.41083797e+00 3.37561781e+00 3.59285585e+00
 2.09506289e-10 2.21103178e-10 1.92663842e-10 2.14380235e-10
 2.10815670e-10 2.36345843e-10 2.07308308e-10 2.30215250e-10
 2.02154449e-10 2.29337257e-10 2.15514639e-10 2.37400621e-10
 4.78577160e+00 4.99235022e+00 4.54228670e+00 4.90413231e+00
 4.85169274e+00 5.322418